In [ ]:
import numpy as np
import os
from keras import optimizers

In [ ]:
from Models.PoolingAndFire import create_model, create_loss_function
from PreProcess import data_generator, get_num_samples
from GenerateData import generate_data

In [ ]:
DATA_DIR = r"./data"

In [ ]:
BATCHSIZE = 16
EPSILON = 1e-16

WEIGHT_DECAY = 0 # 0.001
KEEP_PROB = 0.5
CLASSES = 1

LABEL_WEIGHT = 1.0
OFFSET_LOSS_WEIGHT = 1.0
OFFSET_WEIGHT = 40.0

HEIGHT = 320
WIDTH = 320
CHANNELS = 3

VALIDATION_SPLIT = 0.3

In [ ]:
generate_data(DATA_DIR, WIDTH, HEIGHT, box_min=50, box_max=100, num_images=1000)

In [ ]:
model = create_model(320, 320, 3)
out_shape = model.output_shape
anchor_width = out_shape[1]
anchor_height = out_shape[2]
print(f"Needed anchor shape: {anchor_width}x{anchor_height}")

In [ ]:
model.summary()

In [ ]:
l = create_loss_function(anchor_width,
                         anchor_height,
                         LABEL_WEIGHT,
                         OFFSET_WEIGHT,
                         OFFSET_LOSS_WEIGHT,
                         EPSILON)

In [ ]:
num_samples = get_num_samples(DATA_DIR)
print(f"Number of sampels: {num_samples}")
steps_epoch = num_samples // BATCHSIZE
if steps_epoch < 1:
    steps_epoch = 1
print(f"Steps per epoch: {steps_epoch}")

In [ ]:
opt = optimizers.Adam(lr=1e-4, decay=1e-4)
model.compile(loss=l, optimizer=opt)

In [ ]:
model.fit_generator(data_generator(DATA_DIR, BATCHSIZE, WIDTH, HEIGHT, anchor_width, anchor_height),
                    steps_per_epoch=steps_epoch,
                    epochs=1000,
                    verbose=1)